In [1]:
#importing packages for data manuplation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#for easy access to path
import sys
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.abspath(os.path.join('data')))
sys.path.insert(0,'../scripts_/')

In [3]:
#import packages for machine learning operation
import random
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
from sklearn import tree
import scipy.stats as stat
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingClassifier

#import modules from script
from decision_tree import handler
from decision_tree import DecisionTreesModel
from XGboost import boost
from regression import Logistic

In [4]:
pd.set_option('max_column', None)
pd.set_option('max_rows', None)
#read csv file
df = pd.read_csv('../data/AdSmartABdata.csv')
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [5]:
#Users who responded to the questionaries
answered_df = df.copy(deep=True)
answered_df = answered_df[answered_df['yes'] == 1].append(answered_df[answered_df['no'] == 1])
answered_df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
16,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1,0
45,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,1,0
65,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,exposed,2020-07-09,13,Samsung SM-G935F,6,Facebook,1,0
89,02efdb70-8596-4f3f-b0b2-b91e194f61f7,exposed,2020-07-05,6,Generic Smartphone,6,Chrome Mobile,1,0


In [6]:
yes_df = df[df['yes'] == 1].drop(['yes','no'], axis = 1)
yes_df['response'] = 1
no_df = df[df['no'] == 1].drop(['yes','no'], axis = 1)
no_df['response'] = 0
control_plot_df = yes_df.append(no_df).reset_index(drop=True)
control_plot_df = control_plot_df.drop(columns = ['auction_id'], axis=1)
control_plot_df.head(5)

,experiment,date,hour,device_make,platform_os,browser,response
0,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1
1,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1
2,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,1
3,exposed,2020-07-09,13,Samsung SM-G935F,6,Facebook,1
4,exposed,2020-07-05,6,Generic Smartphone,6,Chrome Mobile,1


>Extracting all who responded either yes or no to one column

### Encoding categorical variables using Label Encoder

In [12]:
def encode_labels(df):
    date_encoder = preprocessing.LabelEncoder()
    device_encoder = preprocessing.LabelEncoder()
    browser_encoder = preprocessing.LabelEncoder()
    experiment_encoder = preprocessing.LabelEncoder()
    response_encoder = preprocessing.LabelEncoder()
    
    df['date'] = date_encoder.fit_transform(control_plot_df['date'])
    df['device_make'] = device_encoder.fit_transform(control_plot_df['device_make'])
    df['browser'] = browser_encoder.fit_transform(control_plot_df['browser'])
    df['experiment'] = experiment_encoder.fit_transform(control_plot_df['experiment'])
    df['browser'] = response_encoder.fit_transform(control_plot_df['browser'])
    df['response'] = response_encoder.fit_transform(control_plot_df['response'])


    
    return df

## Modeling


In [19]:
def feature_data(control_plot_df):
    browser_df = control_plot_df[["experiment", "hour", "date", 'device_make', 'browser', 'response']]
    platfrom_df = control_plot_df[["experiment", "hour", "date", 'device_make', 'platform_os', 'response']]
                                  
    return browser_df, platfrom_df


In [23]:
def save_encoded_df(encoded_df):
    
    browser_df, platfrom_df = feature_data(encoded_df)
    browser_df.to_csv('../data/browser.csv', index=False)
    platfrom_df.to_csv('../data/platform.csv', index=False)

In [24]:
encoded_df = encode_labels(control_plot_df)
save_encoded_df(encoded_df)